In [ ]:
from operator import add
import numpy as np
a=np.array([1,2])
add(a,a)

In [1]:
import os
import sys

# Configure the environment
if 'SPARK_HOME' not in os.environ:
    os.environ['SPARK_HOME'] = '/srv/spark'

# Create a variable for our root path
SPARK_HOME = os.environ['SPARK_HOME']

# Add the PySpark/py4j to the Python Path
sys.path.insert(0, os.path.join(SPARK_HOME, "python", "build"))
sys.path.insert(0, os.path.join(SPARK_HOME, "python"))

In [2]:
#sc.stop()
from pyspark import  SparkContext
sc = SparkContext( 'local[*]', 'pyspark')

In [3]:
def isprime(n):
    """
    check if integer n is a prime
    """
    # make sure n is a positive integer
    n = abs(int(n))
    # 0 and 1 are not primes
    if n < 2:
        return False
    # 2 is the only even prime number
    if n == 2:
        return True
    # all other even numbers are not primes
    if not n & 1:
        return False
    # range starts with 3 and only needs to go up the square root of n
    # for all odd numbers
    for x in range(3, int(n**0.5)+1, 2):
        if n % x == 0:
            return False
    return True

# Create an RDD of numbers from 0 to 1,000,000
nums = sc.parallelize(xrange(10000000),6)

# Compute the number of primes in the RDD
import time
startTime = time.time()
print nums.filter(isprime).count()
print time.time()-startTime

KeyboardInterrupt: 

In [4]:
import csv
from StringIO import StringIO
def split(line):
    reader = csv.reader(StringIO(line))
    return reader.next()

In [5]:
def parse(stringarr):
    return [float(stringarr[x]) for x in range(len(stringarr))]

In [6]:
def spaceBound(x,minBound,maxBound):
    for agentNo in range(len(x)):
        for centroid in range(len(x[agentNo][0])):
            for value in range(len(x[agentNo][0][centroid])):
                if x[agentNo][0][centroid][value]>maxBound[value] or x[agentNo][0][centroid][value] < minBound[value]:
                    x[agentNo][0][centroid][value] = (maxBound[value]-minBound[value])*random.random() + minBound[value]

In [7]:
from operator import add
def clusterDist(agent, dataPoint):
    '''
    having a data point, and an agent, this returns the distance of the data Point to the nearest centroid in the agent
    '''
    dataPoint = np.array(dataPoint)
    mindist = np.linalg.norm(agent[0]-dataPoint)
    for i in range(1,len(agent)):
        mindist = min(mindist,np.linalg.norm(agent[i]-dataPoint))
    return mindist

def evaluateFitness(positions, data):
    icdistance = data.flatMap(lambda dp: list([(i,clusterDist(positions.value[i],dp)) for i in range(len(positions.value))])) 
    fitnessTuple = icdistance.reduceByKey(add).values().collect()
    return np.array(fitnessTuple)

In [8]:
def evalMass(fitness):
    worstfit = max(fitness)
    bestfit = min(fitness)
    total = sum(fitness)
    mass = np.array([(fitness[x]-worstfit)/(bestfit-worstfit) for x in range(len(fitness))])
    mass/=sum(mass)
    return mass

In [9]:
def genForce(agentPair, positions,G):
    agent1 = positions.value[agentPair[0][1]]
    agent2 = positions.value[agentPair[1][1]]
    mass1 = agentPair[0][0]
    mass2 = agentPair[1][0]
    avgctr1 = np.array(reduce(lambda a,b : [a[x] + b[x] for x in range(len(a))], agent1 , [0 for x in range(len(agent1[0]))]))
    avgctr2 = np.array(reduce(lambda a,b : [a[x] + b[x] for x in range(len(a))], agent2 , [0 for x in range(len(agent2[0]))]))
    force = G*mass1*mass2*(avgctr2-avgctr1)/(np.linalg.norm(avgctr2-avgctr1)+ 10**-6)
    return (agentPair[0][1], force)
    
    
    
def gField(mass,positions,G,rNorm, iterCount,maxIter):
    zipmass = zip(mass, [i for i in range(len(mass))])
    sortedmass = sorted(zipmass, key = lambda elem:elem[0])
    finalPer = 2
    kbest = round((finalPer+(1- float(iterCount)/maxIter)*(100-finalPer))*float(len(mass))/100)
    sortedAgents = sc.parallelize(sortedmass)
    sortedAgents.map(lambda pair: (pair[0]*random.random(),pair[1]))
    agentPairs = sortedAgents.cartesian(sc.parallelize(sortedAgents.take(int(kbest))))
    forces = agentPairs.map(lambda pair: genForce(pair,positions,G))
    f = forces.reduceByKey(lambda a,b : [a[x]+b[x] for x in range(len(a))]).values().collect()    
    f = np.array(f)
    a = [f[i]/(mass[i]+10**(-8)) for i in range(len(f))]
    return a
    

In [10]:


def bucketize(data, agent):
    labels = [0 for i in range(len(data))]
    for i in range(len(data)):
        dp = np.array(data[i])
        mindist = sys.maxint/1.0
        currdist=0
        minind=-1
        for j in range(len(agent)):
            centroid = np.array(agent[j])
            currdist = np.linalg.norm(dp-centroid)
            if mindist> currdist:
                mindist = currdist
                minind = j
        labels[i]=minind
    return labels
            
                

def displayPlot(data, bestAgent):
    totaldata = data.collect()
    labels = bucketize(totaldata,bestAgent)
    pos = [list(t) for t in zip(*totaldata)]
    trace = go.Scatter3d(x = pos[0],
                       y = pos[1],
                         z = pos[2],
                       mode = 'markers',
                       marker= dict(color= labels,colorscale='Viridis',size=8,showscale=True))
    plot([trace],show_link=False)

In [17]:
import numpy as np
import random as random
from math import exp
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot,plot
import plotly.plotly as py
import plotly.graph_objs as go


def GSA(dataFilePath, k,N,maxIter,visualize = False):
    if visualize:
        plotly.offline.init_notebook_mode()
    data = sc.textFile(dataFilePath).map(split)
    data = data.map(lambda dp:dp[:4]).map(parse)
    data.cache()
    nod = len(data.first())
    size = data.count()
    x=[[[[0.0 for j in range(nod)] for y in range(k)] , 0.0 , i] for i in range(N)]
    v = [[0.0 for i in range(nod)] for y in range (N)]
    rNorm=2
    alpha=2
    fitness = [0.0 for i in range(N)]
    mass = [0 for i in range (N)]
    bestAgent = [[0 for i in range(nod)] for y in range(k)]
    bestFitness = sys.maxint/1.0
    avgFitnessList = []
    
    #set bounds of data in rangemin and rangemax
    minBound = np.array(data.reduce(lambda a,b:[min(a[i],b[i]) for i in range(len(b))]))
    print "minimum boundaries of data : " + str(minBound)
    maxBound = np.array(data.reduce(lambda a,b:[max(a[i],b[i]) for i in range(len(b))]))
    print "maximum boundaries of data : " + str(maxBound)
    
    #evaluate value of g0
    g0 = (np.min(maxBound-minBound))/(N**0.3)
    print "choosing g0 as " + str(g0)
    #randomly initialise agent populations x between maxBound and minBound
    for agentNo in range(len(x)):
        for centroid in range(len(x[agentNo][0])):
            for value in range(len(x[agentNo][0][centroid])):
                x[agentNo][0][centroid][value] = (maxBound[value]-minBound[value])*random.random() + minBound[value]
    
    for iterCount in range(maxIter):
        #TODO:write spaceBound function
        spaceBound(x,minBound,maxBound)
        positions = sc.broadcast([list(t) for t in zip(*x)][0])
        
        fitness = evaluateFitness(positions,data)
        #print "fitness of all agents: "
        #print fitness
        
        #evaluate average fitness, and globally best agent till now
        avgFitness = np.average(fitness)
        avgFitnessList.append(avgFitness)
        localMaxFitness = np.min(fitness)
        if localMaxFitness < bestFitness:
            bestFitness= localMaxFitness
            bestAgent = x[np.argmax(fitness)][0]
            
                
        print "best fitness yet is " + str(bestFitness) + " and average is " + str(avgFitness)
        
        mass = evalMass(fitness)
        #print "mass of all agents : " + str(mass)
        #add this fitness data to the agents
        for i in range(len(x)):
            x[i][1] = mass[i]
        
        G = g0*exp(-alpha*iterCount/maxIter)
        a = gField(mass,positions,G,rNorm, iterCount,maxIter)

        #update velocity
        for agent in range(len(v)):
            for dim in range(len(v[agent])):
                try:
                    v[agent][dim] = v[agent][dim]*random.random() + a[agent][dim]
                except:
                    pass
        #print "updated velocity : "
        #print v
        #print "average movement: " + str(np.average(v))
        
        #update positions
        for agent in range(len(x)):
            for centroid in range(len(x[agent][0])):
                for dim in range(len(x[agent][0][centroid])):
                    x[agent][0][centroid][dim] += v[agent][dim]
        #print "updated positions : "
        #for agent in range(len(x)):
        #    print x[agent][0]
        
    if visualize==True:
        displayPlot(data,bestAgent)
        
    
    return bestAgent
    

In [19]:
path = r'/Users/Jai/Documents/code/python-learning/datasets/iris.data'
bestsol = GSA(path,3,5000,100,visualize=True)

minimum boundaries of data : [ 4.3  2.   1.   0.1]
maximum boundaries of data : [ 7.9  4.4  6.9  2.5]
choosing g0 as 0.742198787306
best fitness yet is 195.420858121 and average is 301.693405004
best fitness yet is 181.291895691 and average is 291.999215683
best fitness yet is 181.291895691 and average is 286.32138463
best fitness yet is 173.234307264 and average is 287.903422915
best fitness yet is 173.234307264 and average is 291.344469661
best fitness yet is 173.234307264 and average is 294.455765703
best fitness yet is 173.234307264 and average is 298.625227836
best fitness yet is 173.234307264 and average is 294.419545421
best fitness yet is 173.234307264 and average is 296.936627231
best fitness yet is 173.234307264 and average is 294.509987328
best fitness yet is 173.234307264 and average is 296.659041176
best fitness yet is 173.234307264 and average is 298.371528667
best fitness yet is 173.234307264 and average is 300.971675125
best fitness yet is 173.234307264 and average is 3

KeyboardInterrupt: 

In [ ]:
path = r'/Users/Jai/Documents/code/python-learning/datasets/iris.data'
data = sc.textFile(path).map(split)
data = data.map(lambda dp:dp[:4]).map(parse)
print data.first()
data.cache()
minBound = data.reduce(lambda a,b:[min(a[x],b[x]) for x in range(int(len(b)))])
maxBound = data.reduce(lambda a,b:[max(a[x],b[x]) for x in range(int(len(b)))])
print minBound
print maxBound

In [ ]:
f = [[1,2],[3,4]]
f = np.array(f)
np.average(f)